In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import spacy
import numpy as np
nlp = spacy.load("en_core_web_sm")

In [0]:
import pandas as pd
def read_file(filepath):
  df=pd.read_table(filepath,sep=' ')
  if('POS_TAGS' in df.columns):
    df.drop(['POS_TAGS'],inplace=True,axis=1)
  return df 

In [0]:
df_ug=read_file('/content/drive/My Drive/IBM_Debater_(R)_SC_COLING_2018/LEXICON_UG.txt')
df_bg=read_file('/content/drive/My Drive/IBM_Debater_(R)_SC_COLING_2018/LEXICON_BG.txt')

In [0]:
reversers=pd.read_excel('/content/drive/My Drive/IBM_Debater_(R)_SC_COLING_2018/SEMANTIC_CLASSES.xlsx',sheet_name=['REVERSER_POS','REVERSER_NEG'])

In [0]:
pos_reversers=list(reversers['REVERSER_POS'].items())
pos_reversers=list(pos_reversers[0][1])
pos_reversers.append(reversers['REVERSER_POS'].keys()[0])
# pos_reversers[0][0].extend()
neg_reversers=list(reversers['REVERSER_NEG'].items())
neg_reversers=list(neg_reversers[0][1])
neg_reversers.append(reversers['REVERSER_NEG'].keys()[0])

In [21]:
!pip install git+https://github.com/huggingface/neuralcoref.git
import nltk
nltk.download('punkt')
import neuralcoref

def polarity(document,imptwords=None):
  nlp = spacy.load("en_core_web_sm")
  neuralcoref.add_to_pipe(nlp)
  doc=nlp(document)
  dict_ents={}
  for ent in doc.ents:
    if(ent.label_ in ['ORG','PERSON','NORP']):
      dict_ents[ent.text]=ent.label_
  result=[]
  # print(dict_ents)
  
  for sent in nltk.sent_tokenize(doc._.coref_resolved):
    if(imptwords is None or sum([1 for word in imptwords if word in str(sent)])>0):
      doc=nlp(str(sent))
      chunks=[]
      for chunk in doc.noun_chunks:
        # print(chunk)
        chunks.append((chunk.text, chunk.root.dep_,chunk.root.head.text))
      sent_subj=None
      sent_obj=None
      polarity=0
      for chunk in reversed(chunks):        
        # key=[key for key in dict_ents.keys() if key in chunk[0]]
        if(chunk[1]in ['dobj']):
          sent_obj=chunk[0]
        elif(chunk[1]in ['nsubj']):
          sent_subj=chunk[0]
      # bigrams=list(nltk.bigrams(reversed(str(sent).split())))
      sent_tokenized=list(reversed(sent.strip('.').split()))
      bigrams=[]
      for i in range(len(sent_tokenized)):
        try:
          bigrams.append((sent_tokenized[i],sent_tokenized[i+1]))
          i+=2
        except:
          break 
      for t1,t2 in bigrams:
        t1=t1.replace('\"','')
        t2=t2.replace('\"','')
        itm=df_bg[df_bg['BIGRAM']==str(t1)+'-'+str(t2)]
        if len(itm)>0:
          polarity+=float(itm['SENTIMENT_SCORE'])
        elif(t2 in pos_reversers):
          itm=df_ug[df_ug['UNIGRAM']==str(t1)]          
          if len(itm)>0:
            val=float(itm['SENTIMENT_SCORE'])
            if(val<0):
              polarity+= -1*float(itm['SENTIMENT_SCORE'])
        elif(t1 in neg_reversers):
          itm=df_ug[df_ug['UNIGRAM']==str(t1)]          
          if len(itm)>0:
            val=float(itm['SENTIMENT_SCORE'])
            if(val>0):
              polarity+= -1*float(itm['SENTIMENT_SCORE'])      
        else:
          itm=df_ug[df_ug['UNIGRAM']==str(t1)]          
          if len(itm)>0:
            polarity+= float(itm['SENTIMENT_SCORE'])
          itm=df_ug[df_ug['UNIGRAM']==str(t2)]
          if len(itm)>0:
            polarity+= float(itm['SENTIMENT_SCORE'])
      result.append((sent_subj,sent_obj,polarity)) 
  return result,dict_ents       

  Cloning https://github.com/huggingface/neuralcoref.git to /tmp/pip-req-build-bz4l24a3
  Running command git clone -q https://github.com/huggingface/neuralcoref.git /tmp/pip-req-build-bz4l24a3
  Created wheel for neuralcoref: filename=neuralcoref-4.0-cp36-cp36m-linux_x86_64.whl size=896568 sha256=a36e879671037f436a6a72af3e894ad358726c88a4bdc7524356fd158e572ae5
  Stored in directory: /tmp/pip-ephem-wheel-cache-328moz54/wheels/df/d8/1e/7be05e66e616d681362efab983782070a0423dfd74d2ea280d
Successfully built neuralcoref
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def read_file_content(path):
  with open(path,'r') as f:
    text=f.read()
  return text  

In [0]:
def compare(w1,w2):
  for w in w1.split():
    if(w in w2.split()):
      return True
  return False

def bigram_compare(w1,w2):
  w1_bigrams=list(nltk.bigrams(w1.split()))
  w2_bigrams=list(nltk.bigrams(w2.split()))
  if(len(w1_bigrams)==0 or len(w2_bigrams)==0):
    return compare(w1,w2)
  for tok1,tok2 in w1_bigrams:
    for t1,t2 in w2_bigrams:
      if(tok1==t1 and tok2==t2):
        return True
  return False            

In [0]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)
pronouns=['i','he','she','you','we','they','them','it','his','who']

def document_polarity(doc):
    # doc=read_file_content('/content/drive/My Drive/005.txt')
    wordcloud = WordCloud(
            background_color='white',
            stopwords=stopwords,
            max_words=5,
            max_font_size=40, 
            scale=3,
            random_state=1
        ).generate(str(doc))
    imptwords=[k for k,v in wordcloud.words_.items()]   

    scores,dict_ents=polarity(doc,imptwords)
    # print(scores)
    refined=[]
    for subj,obj,score in scores:
      if(subj is not None and obj is not None):#and sum([1 for key in dict_ents.keys() if (compare(subj,key) and compare(obj,key))])>0):
        if((sum([1 for w2 in imptwords if(compare(subj,w2))])>0 or sum([1 for w2 in imptwords if(compare(obj,w2))])>0)
          and (subj.lower() not in pronouns) and (obj.lower() not in pronouns)):
          refined.append((subj,obj,score))
      elif(subj is None and obj is not None and obj.lower() not in pronouns and sum([1 for w2 in imptwords if(compare(obj,w2))])>0):
        subj='author'
        refined.append((subj,obj,score))           
    return imptwords,refined
# polarity('The Tories dismissed the maternity pay plan as "desperate", while the Liberal Democrats said it was misdirected')

In [0]:
def sentence_polarity(doc): 
    scores,dict_ents=polarity(doc)
    refined=[]
    for subj,obj,score in scores:
      if(subj is None and obj is not None and obj.lower() not in pronouns):
          subj='author'
      if(obj is not None and subj.lower() not in pronouns and obj.lower() not in pronouns):    
          refined.append((subj,obj,score))           
    return refined

In [0]:
import pickle
with open('/content/drive/My Drive/documents.pkl','rb') as f:
  documents=pickle.load(f)
with open('/content/drive/My Drive/data.pkl','rb') as f:
  data=pickle.load(f)    

In [27]:
file_names=list(data['file_name'].unique())
unigrams=[]
bigrams=[]
for index,doc in enumerate(documents):
  imptwords,sent_doc=document_polarity(doc)
  sent_doc_gt=data[data['file_name']==file_names[index]]
  print(imptwords)
  print(sent_doc)
  refined=[]
  for indx,row in sent_doc_gt.iterrows():
    if((sum([1 for w2 in imptwords if(compare(row['agent'],w2))])>0 or sum([1 for w2 in imptwords if(compare(row['target'],w2))])>0)
          and (row['agent'].lower().strip() not in pronouns) and (row['target'].lower().strip() not in pronouns)
          and ('pos' in row['sentiment_type'] or 'neg' in row['sentiment_type'])):
          refined.append((row['agent'],row['target'],row['sentiment_type'],row['sentiment_intensity']))
  # Dircard findings where agent is same as target
  # 1 token match between agent and target of predicted and ground truth
  # score is matched polarity items/total items in ground truth
  print(refined)
  match=0
  for pred_agent,pred_target,score in sent_doc:
    for agent,target,sent_type,sent_intensity in refined:
      if(pred_agent.strip()!=pred_target.strip() and compare(agent,pred_agent)
        and compare(target,pred_target) and ((score>0 and 'pos' in sent_type) or (score<0 and 'neg' in sent_type))):
        match+=1
        break
  bigram_match=0      
  for pred_agent,pred_target,score in sent_doc:
    for agent,target,sent_type,sent_intensity in refined:
      if(pred_agent.strip()!=pred_target.strip() and bigram_compare(agent,pred_agent)
        and bigram_compare(target,pred_target) and ((score>0 and 'pos' in sent_type) or (score<0 and 'neg' in sent_type))):
        bigram_match+=1
        break 
  if(len(refined)>0):
    unigrams.append(match/len(refined)) 
    bigrams.append(bigram_match/len(refined))           
    print('unigram match {}'.format(match/len(refined)))
    print('bigram match {}'.format(bigram_match/len(refined)))     
  print('============================================================================')
print('average unigram score: {}'.format(np.mean(unigrams)))
print('average bigram score: {}'.format(np.mean(bigrams)))  

['US', 'United States', 'war', 'Bush', 'Missile']
[('Bush', 'the plan', 0.1691692), ('Bush', 'a threat', -0.9687328720000001), ('Bush', 'NMD', -0.08523259999999988), ('Bush', 'funds', 0.4966638600000002), ('the United States', 'a fifth missile interception test', 0.44418230600000036), ('the US development', 'other countries', 2.521305939999999), ('The Sino-US plane collision event', 'the US alternate application', 0.2772025680000001), ('the US side', 'all the US crew members', -1.6726829059999992), ('the US negotiation representative', 'the implementation', 0.7416281360000001), ('the Bush administration', 'the international treaties', 0.6429173080000001), ('the Bush administration', 'anything', -0.12624752), ('the Bush administration', 'anything', -0.819497972), ('the United States', 'the "war', -0.4731002000000002), ('the United States', 'the United States', 0.98129572), ('the United States', 'pragmatism', 0.3747796), ('the United States', 'the United States distance', -0.777394792), 

In [33]:
file_names=list(data['file_name'].unique())

nlp = spacy.load("en_core_web_sm")
neuralcoref.add_to_pipe(nlp)
unigrams=[]
bigrams=[]
for index,doc in enumerate(documents):  
  docx=nlp(doc)
  sents=nltk.sent_tokenize(docx._.coref_resolved)
  sent_doc_gt=data[data['file_name']==file_names[index]]
  refined=[]
  for indx,row in sent_doc_gt.iterrows():
    if((row['agent'].lower().strip() not in pronouns) and (row['target'].lower().strip() not in pronouns)
          and ('pos' in row['sentiment_type'] or 'neg' in row['sentiment_type'])):
          refined.append((row['agent'],row['target'],row['sentiment_type'],row['sentiment_intensity']))
  print(refined)        
  for indx,sent in enumerate(sents):
    sent_doc=sentence_polarity(sent)
    if(len(sent_doc)==0):
      continue
    print(sent_doc)
    match=0
    for pred_agent,pred_target,score in sent_doc:
      for agent,target,sent_type,sent_intensity in refined:
        if(pred_agent.strip()!=pred_target.strip() and compare(agent,pred_agent)
          and compare(target,pred_target) and ((score>0 and 'pos' in sent_type) or (score<0 and 'neg' in sent_type))):
          match+=1
          break
    bigram_match=0      
    for pred_agent,pred_target,score in sent_doc:
      for agent,target,sent_type,sent_intensity in refined:
        if(pred_agent.strip()!=pred_target.strip() and bigram_compare(agent,pred_agent)
          and bigram_compare(target,pred_target) and ((score>0 and 'pos' in sent_type) or (score<0 and 'neg' in sent_type))):
          bigram_match+=1
          break 
    if(len(refined)>0): 
      unigrams.append(match/len(refined)) 
      bigrams.append(bigram_match/len(refined))            
      print('unigram match {}'.format(match/len(refined)))
      print('bigram match {}'.format(bigram_match/len(refined)))     
  print('============================================================================')  
print('average unigram score: {}'.format(np.mean(unigrams)))
print('average bigram score: {}'.format(np.mean(bigrams)))   

[('the United States ', 'the trend of pragmatism in its foreign policy ', 'arguing-pos', 'medium'), ('author', 'Pragmatism ', 'arguing-pos', 'low-medium'), ('author', 'Alternate use of and complementary nature of pragmatism and unilateralism ', 'arguing-pos', 'low-medium'), ('Bush ', 'deploying the National Missile Defense System ', 'sentiment-pos', 'low-medium'), ('The international community and even US western allies, including US neighbor Canada ', 'the plan ', 'intention-pos', 'low-medium'), ('author', 'Bush ', 'sentiment-neg', 'medium-high'), ('Russia ', 'revise the Anti-Ballistic Missile Treaty ', 'sentiment-neg', 'medium'), ('the Soviet Union and the United States ', 'the Anti-Ballistic Missile Treaty signed by the Soviet Union and the United States in 1972 ', 'arguing-pos', 'medium'), ('author', 'the US ', 'sentiment-neg', 'high-extreme'), ('author', 'the US ', 'sentiment-neg', 'low-medium'), ('author', 'the US  logic ', 'sentiment-neg', 'medium-high'), ('author', 'the US se '

In [26]:
# Considering only those results where only named entities are present as targets
nlp = spacy.load("en_core_web_sm")
neuralcoref.add_to_pipe(nlp) 

for index,doc in enumerate(documents):
  docx=nlp(doc)
  ents=[]
  for ent in docx.ents:
    if(ent.label_ in ['ORG','PERSON','NORP']):
      ents.append(ent.text)  

  imptwords,sent_doc=document_polarity(doc)
  sent_doc_gt=data[data['file_name']==data.iloc[index]['file_name']]
  print(imptwords)
  # print(sent_doc)
  refined=[]
  for indx,row in sent_doc_gt.iterrows():
    if((sum([1 for w2 in imptwords if(compare(row['agent'],w2))])>0 or sum([1 for w2 in imptwords if(compare(row['target'],w2))])>0)
          and (row['agent'].lower().strip() not in pronouns) and (row['target'].lower().strip() not in pronouns)
          and ('pos' in row['sentiment_type'] or 'neg' in row['sentiment_type'])):
          refined.append((row['agent'],row['target'],row['sentiment_type'],row['sentiment_intensity']))
  # Dircard findings where agent is same as target
  # 1 token match between agent and target of predicted and ground truth
  # score is matched polarity items/total items in ground truth
  print(refined)
  match=0
  pred_with_ents=[]
  for pred_agent,pred_target,score in sent_doc:
    if(sum([1 for word in pred_target.split() if word in ents])>0):
        pred_with_ents.append((pred_agent,pred_target,score))
        for agent,target,sent_type,sent_intensity in refined:
          if(pred_agent.strip()!=pred_target.strip() and compare(agent,pred_agent)
            and compare(target,pred_target) and ((score>0 and 'pos' in sent_type) or (score<0 and 'neg' in sent_type))):
            match+=1
            break
  print(pred_with_ents)          
  bigram_match=0      
  for pred_agent,pred_target,score in pred_with_ents:
    for agent,target,sent_type,sent_intensity in refined:
      if(pred_agent.strip()!=pred_target.strip() and bigram_compare(agent,pred_agent)
        and bigram_compare(target,pred_target) and ((score>0 and 'pos' in sent_type) or (score<0 and 'neg' in sent_type))):
        bigram_match+=1
        break 
  if(len(refined)>0):           
    print('unigram match {}'.format(match/len(refined)))
    print('bigram match {}'.format(bigram_match/len(refined)))     
  print('============================================================================')    

['US', 'United States', 'war', 'Bush', 'Missile']
[('the United States ', 'the trend of pragmatism in its foreign policy ', 'arguing-pos', 'medium'), ('Bush ', 'deploying the National Missile Defense System ', 'sentiment-pos', 'low-medium'), ('The international community and even US western allies, including US neighbor Canada ', 'the plan ', 'intention-pos', 'low-medium'), ('author', 'Bush ', 'sentiment-neg', 'medium-high'), ('Russia ', 'revise the Anti-Ballistic Missile Treaty ', 'sentiment-neg', 'medium'), ('the Soviet Union and the United States ', 'the Anti-Ballistic Missile Treaty signed by the Soviet Union and the United States in 1972 ', 'arguing-pos', 'medium'), ('author', 'the US ', 'sentiment-neg', 'high-extreme'), ('author', 'the US ', 'sentiment-neg', 'low-medium'), ('author', 'the US  logic ', 'sentiment-neg', 'medium-high'), ('author', 'the US se ', 'sentiment-neg', 'low-medium'), ('The US ', 'US crew members ', 'arguing-pos', 'low-medium'), ('major US political figures 

KeyboardInterrupt: ignored